In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from collections import Counter
from imblearn.over_sampling import ADASYN
adasyn= ADASYN(random_state= 15)

X_adasyn, y_adasyn= adasyn.fit_resample(X, y)
print(f'ADASYN: {Counter(y_adasyn)}')

ADASYN: Counter({0: 42582, 1: 41291})


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

X_adasyn_train, y_adasyn_train= adasyn.fit_resample(X_train, y_train)

--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
lr_adasyn= LogisticRegression(random_state=15)
lr_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_lr_adasyn= lr_adasyn.predict(X_test)
y_pred_prob_lr_adasyn= lr_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_adasyn, labels= lr_adasyn.classes_)
print(classification_report(y_test, y_pred_lr_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr_adasyn))

              precision    recall  f1-score   support

           0       0.97      0.73      0.83      8517
           1       0.24      0.82      0.37       912

    accuracy                           0.73      9429
   macro avg       0.61      0.77      0.60      9429
weighted avg       0.90      0.73      0.79      9429

Precision: 0.2426208238728511
Accuracy: 0.734966592427617
AUC: 0.8511940257771351


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt_adasyn= DecisionTreeClassifier(random_state=15)
dt_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_dt_adasyn= dt_adasyn.predict(X_test)
y_pred_prob_dt_adasyn= dt_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_adasyn, labels= dt_adasyn.classes_)
print(classification_report(y_test, y_pred_dt_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_adasyn))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      8517
           1       0.23      0.26      0.24       912

    accuracy                           0.85      9429
   macro avg       0.58      0.58      0.58      9429
weighted avg       0.85      0.85      0.85      9429

Precision: 0.23221343873517786
Accuracy: 0.8457948881111464
AUC: 0.5822643284123188


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_adasyn= GridSearchCV(estimator= dt_adasyn, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_adasyn.fit(X_adasyn_train, y_adasyn_train)

print("Best Parameters:", gs_dt_adasyn.best_params_)
print("Best Precision Score:", gs_dt_adasyn.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 10, 'min_samples_split': 2}
Best Precision Score: 0.9398972639759451


In [12]:
tuned_dt_adasyn= gs_dt_adasyn.best_estimator_
y_pred_tuned_dt_adasyn= tuned_dt_adasyn.predict(X_test)
y_pred_prob_tuned_dt_adasyn= tuned_dt_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_adasyn, labels= tuned_dt_adasyn.classes_)
print(classification_report(y_test, y_pred_tuned_dt_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_adasyn))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93      8517
           1       0.31      0.22      0.26       912

    accuracy                           0.88      9429
   macro avg       0.61      0.59      0.60      9429
weighted avg       0.86      0.88      0.87      9429

Precision: 0.30597014925373134
Accuracy: 0.875702619577898
AUC: 0.718136160599338


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn_adasyn= KNeighborsClassifier()
knn_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_knn_adasyn= knn_adasyn.predict(X_test)
y_pred_prob_knn_adasyn= knn_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_knn_adasyn, labels= knn_adasyn.classes_)
print(classification_report(y_test, y_pred_knn_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn_adasyn))

              precision    recall  f1-score   support

           0       0.95      0.77      0.85      8517
           1       0.22      0.61      0.32       912

    accuracy                           0.75      9429
   macro avg       0.58      0.69      0.58      9429
weighted avg       0.88      0.75      0.80      9429

Precision: 0.21672555948174324
Accuracy: 0.7502386255170219
AUC: 0.7393989755267587


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn_adasyn= GridSearchCV(estimator= knn_adasyn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn_adasyn.fit(X_adasyn_train, y_adasyn_train)
print("Best Parameters:", gs_knn_adasyn.best_params_)
print("Best Precision Score:", gs_knn_adasyn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.9257162034464381


In [15]:
tuned_knn_adasyn= gs_knn_adasyn.best_estimator_
y_pred_tuned_knn_adasyn= tuned_knn_adasyn.predict(X_test)
y_pred_prob_tuned_knn_adasyn= tuned_knn_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn_adasyn, labels= tuned_knn_adasyn.classes_)
print(classification_report(y_test, y_pred_tuned_knn_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn_adasyn))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      8517
           1       0.25      0.19      0.22       912

    accuracy                           0.86      9429
   macro avg       0.58      0.56      0.57      9429
weighted avg       0.85      0.86      0.86      9429

Precision: 0.24684431977559607
Accuracy: 0.8649909852582458
AUC: 0.6207608003806628


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb_adasyn= GaussianNB()
nb_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_nb_adasyn= nb_adasyn.predict(X_test)
y_pred_prob_nb_adasyn= nb_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_nb_adasyn, labels= nb_adasyn.classes_)
print(classification_report(y_test, y_pred_nb_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb_adasyn))

              precision    recall  f1-score   support

           0       0.98      0.52      0.68      8517
           1       0.17      0.89      0.28       912

    accuracy                           0.56      9429
   macro avg       0.57      0.70      0.48      9429
weighted avg       0.90      0.56      0.64      9429

Precision: 0.16550594018844736
Accuracy: 0.556898928836568
AUC: 0.8010708459242506


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc_adasyn= SVC(kernel= 'rbf', gamma= 1, probability= True, random_state=15)
svc_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_svc_adasyn= svc_adasyn.predict(X_test)
y_pred_prob_svc_adasyn= svc_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_adasyn, labels= svc_adasyn.classes_)
print(classification_report(y_test, y_pred_svc_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_adasyn))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      8517
           1       0.15      0.04      0.06       912

    accuracy                           0.89      9429
   macro avg       0.53      0.51      0.50      9429
weighted avg       0.83      0.89      0.85      9429

Precision: 0.15486725663716813
Accuracy: 0.8867324212535794
AUC: 0.6988701261048594


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_adasyn= RandomForestClassifier(random_state=15)
rf_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_rf_adasyn= rf_adasyn.predict(X_test)
y_pred_prob_rf_adasyn= rf_adasyn.predict_proba(X_test)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf_adasyn, labels= rf_adasyn.classes_)
print(classification_report(y_test, y_pred_rf_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_adasyn))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8517
           1       0.37      0.11      0.17       912

    accuracy                           0.90      9429
   macro avg       0.64      0.55      0.56      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.36619718309859156
Accuracy: 0.8952168840810266
AUC: 0.8208678103030266


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_adasyn= GridSearchCV(estimator= rf_adasyn, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_adasyn.fit(X_adasyn_train, y_adasyn_train)
print("Best Parameters:", gs_tuned_rf_adasyn.best_params_)
print("Best Precision Score:", gs_tuned_rf_adasyn.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Best Precision Score: 0.9767393547574545


In [21]:
tuned_rf_adasyn= gs_tuned_rf_adasyn.best_estimator_
y_pred_tuned_rf_adasyn= tuned_rf_adasyn.predict(X_test)
y_pred_prob_tuned_rf_adasyn= tuned_rf_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_adasyn, labels= tuned_rf_adasyn.classes_)
print(classification_report(y_test, y_pred_tuned_rf_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_adasyn))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.43      0.12      0.19       912

    accuracy                           0.90      9429
   macro avg       0.67      0.55      0.57      9429
weighted avg       0.87      0.90      0.87      9429

Precision: 0.42696629213483145
Accuracy: 0.8991409481387209
AUC: 0.8327227768405399


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada_adasyn= AdaBoostClassifier(random_state=15)
ada_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_ada_adasyn= ada_adasyn.predict(X_test)
y_pred_prob_ada_adasyn= ada_adasyn.predict_proba(X_test)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada_adasyn, labels= ada_adasyn.classes_)
print(classification_report(y_test, y_pred_ada_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_adasyn))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93      8517
           1       0.38      0.45      0.41       912

    accuracy                           0.88      9429
   macro avg       0.66      0.68      0.67      9429
weighted avg       0.89      0.88      0.88      9429

Precision: 0.3776744186046512
Accuracy: 0.8753844522218687
AUC: 0.8379871449052361


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_adasyn= GridSearchCV(estimator= ada_adasyn, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_adasyn.fit(X_adasyn_train, y_adasyn_train)
print("Best Parameters:", gs_ada_adasyn.best_params_)
print("Best Precision Score:", gs_ada_adasyn.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9655599496326331


In [25]:
tuned_ada_adasyn= gs_ada_adasyn.best_estimator_
y_pred_tuned_ada_adasyn= tuned_ada_adasyn.predict(X_test)
y_pred_prob_tuned_ada_adasyn= tuned_ada_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_adasyn, labels= tuned_ada_adasyn.classes_)
print(classification_report(y_test, y_pred_tuned_ada_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_adasyn))

              precision    recall  f1-score   support

           0       0.92      0.97      0.95      8517
           1       0.47      0.26      0.34       912

    accuracy                           0.90      9429
   macro avg       0.70      0.62      0.64      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.46862745098039216
Accuracy: 0.8998833386361226
AUC: 0.8481449124454908


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb_adasyn= GradientBoostingClassifier(random_state=15)
grb_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_grb_adasyn= grb_adasyn.predict(X_test)
y_pred_prob_grb_adasyn= grb_adasyn.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_adasyn, labels= grb_adasyn.classes_)
print(classification_report(y_test, y_pred_grb_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_adasyn))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      8517
           1       0.42      0.32      0.36       912

    accuracy                           0.89      9429
   macro avg       0.68      0.64      0.65      9429
weighted avg       0.88      0.89      0.89      9429

Precision: 0.42219020172910665
Accuracy: 0.8918230989500477
AUC: 0.8444009169483531


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_adasyn= GridSearchCV(estimator= grb_adasyn, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_adasyn.fit(X_adasyn_train, y_adasyn_train)

print("Best Parameters:", gs_grb_adasyn.best_params_)
print("Best Precision Score:", gs_grb_adasyn.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.9742661570580525


In [28]:
tuned_grb_adasyn= gs_grb_adasyn.best_estimator_
y_pred_tuned_grb_adasyn= tuned_grb_adasyn.predict(X_test)
y_pred_prob_tuned_grb_adasyn= tuned_grb_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_adasyn, labels= tuned_grb_adasyn.classes_)
print(classification_report(y_test, y_pred_tuned_grb_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_adasyn))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.48      0.18      0.26       912

    accuracy                           0.90      9429
   macro avg       0.70      0.58      0.60      9429
weighted avg       0.88      0.90      0.88      9429

Precision: 0.48214285714285715
Accuracy: 0.9020044543429844
AUC: 0.8464096059686613


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb_adasyn= XGBClassifier(random_state=15)
xgb_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_xgb_adasyn= xgb_adasyn.predict(X_test)
y_pred_prob_xgb_adasyn= xgb_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_adasyn, labels= xgb_adasyn.classes_)
print(classification_report(y_test, y_pred_xgb_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_adasyn))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.45      0.13      0.21       912

    accuracy                           0.90      9429
   macro avg       0.68      0.56      0.58      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.45018450184501846
Accuracy: 0.900413617562838
AUC: 0.8344932941135274


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_adasyn= GridSearchCV(estimator= xgb_adasyn, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_adasyn.fit(X_adasyn_train, y_adasyn_train)

print("Best Parameters:", gs_xgb_adasyn.best_params_)
print("Best Precision Score:", gs_xgb_adasyn.best_score_)

tuned_xgb_adasyn= gs_xgb_adasyn.best_estimator_
y_pred_tuned_xgb_adasyn= tuned_xgb_adasyn.predict(X_test)
y_pred_prob_tuned_xgb_adasyn= tuned_xgb_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_adasyn, labels= tuned_xgb_adasyn.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_adasyn))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.9715141447257174
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.46      0.20      0.28       912

    accuracy                           0.90      9429
   macro avg       0.69      0.59      0.61      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.4619289340101523
Accuracy: 0.9000954502068088
AUC: 0.8446253133567746


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm_adasyn= LGBMClassifier(random_state=15)
lgm_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_lgm_adasyn= lgm_adasyn.predict(X_test)
y_pred_prob_lgm_adasyn= lgm_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_adasyn, labels= lgm_adasyn.classes_)
print(classification_report(y_test, y_pred_lgm_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_adasyn))

[LightGBM] [Info] Number of positive: 34618, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026194 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8604
[LightGBM] [Info] Number of data points in the train set: 68683, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504026 -> initscore=0.016103
[LightGBM] [Info] Start training from score 0.016103
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      8517
           1       0.53      0.15      0.24       912

    accuracy                           0.91      9429
   macro avg       0.72      0.57      0.59      9429
weighted avg       0.88      0.91      0.88      9429

Precision: 0.5340909090909091
Accuracy: 0.9051861279032771
AUC: 0.8464531849613467


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_adasyn= GridSearchCV(estimator= lgm_adasyn, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_adasyn.fit(X_adasyn_train, y_adasyn_train)

print("Best Parameters:", gs_lgm_adasyn.best_params_)
print("Best Precision Score:", gs_lgm_adasyn.best_score_)

tuned_lgm_adasyn= gs_lgm_adasyn.best_estimator_
y_pred_tuned_lgm_adasyn= tuned_lgm_adasyn.predict(X_test)
y_pred_prob_tuned_lgm_adasyn= tuned_lgm_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_adasyn, labels= tuned_lgm_adasyn.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_adasyn))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 34618, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008377 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8604
[LightGBM] [Info] Number of data points in the train set: 68683, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504026 -> initscore=0.016103
[LightGBM] [Info] Start training from score 0.016103
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.5}
Best Precision Score: 0.98109715022581

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat_adasyn= CatBoostClassifier(random_state=15)
cat_adasyn.fit(X_adasyn_train, y_adasyn_train)
y_pred_cat_adasyn= cat_adasyn.predict(X_test)
y_pred_prob_cat_adasyn= cat_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_adasyn, labels= cat_adasyn.classes_)
print(classification_report(y_test, y_pred_cat_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_adasyn))

Learning rate set to 0.0627
0:	learn: 0.6234886	total: 169ms	remaining: 2m 49s
1:	learn: 0.5708888	total: 190ms	remaining: 1m 35s
2:	learn: 0.5232652	total: 212ms	remaining: 1m 10s
3:	learn: 0.4785892	total: 233ms	remaining: 57.9s
4:	learn: 0.4499429	total: 253ms	remaining: 50.4s
5:	learn: 0.4253189	total: 274ms	remaining: 45.3s
6:	learn: 0.3989221	total: 294ms	remaining: 41.7s
7:	learn: 0.3826662	total: 316ms	remaining: 39.1s
8:	learn: 0.3661309	total: 338ms	remaining: 37.2s
9:	learn: 0.3540363	total: 362ms	remaining: 35.8s
10:	learn: 0.3435413	total: 384ms	remaining: 34.5s
11:	learn: 0.3316176	total: 407ms	remaining: 33.5s
12:	learn: 0.3186954	total: 432ms	remaining: 32.8s
13:	learn: 0.3085411	total: 456ms	remaining: 32.1s
14:	learn: 0.3002819	total: 479ms	remaining: 31.5s
15:	learn: 0.2946847	total: 505ms	remaining: 31.1s
16:	learn: 0.2885215	total: 531ms	remaining: 30.7s
17:	learn: 0.2806504	total: 557ms	remaining: 30.4s
18:	learn: 0.2709597	total: 584ms	remaining: 30.1s
19:	learn:

162:	learn: 0.1541891	total: 4.31s	remaining: 22.1s
163:	learn: 0.1541095	total: 4.33s	remaining: 22.1s
164:	learn: 0.1540329	total: 4.35s	remaining: 22s
165:	learn: 0.1539570	total: 4.36s	remaining: 21.9s
166:	learn: 0.1538554	total: 4.38s	remaining: 21.9s
167:	learn: 0.1532092	total: 4.4s	remaining: 21.8s
168:	learn: 0.1531235	total: 4.42s	remaining: 21.7s
169:	learn: 0.1530346	total: 4.44s	remaining: 21.7s
170:	learn: 0.1529780	total: 4.45s	remaining: 21.6s
171:	learn: 0.1527913	total: 4.47s	remaining: 21.5s
172:	learn: 0.1525919	total: 4.49s	remaining: 21.5s
173:	learn: 0.1525283	total: 4.51s	remaining: 21.4s
174:	learn: 0.1524536	total: 4.53s	remaining: 21.3s
175:	learn: 0.1524100	total: 4.54s	remaining: 21.3s
176:	learn: 0.1523602	total: 4.57s	remaining: 21.2s
177:	learn: 0.1522784	total: 4.59s	remaining: 21.2s
178:	learn: 0.1521831	total: 4.61s	remaining: 21.1s
179:	learn: 0.1521176	total: 4.62s	remaining: 21.1s
180:	learn: 0.1517275	total: 4.64s	remaining: 21s
181:	learn: 0.151

324:	learn: 0.1357877	total: 8.37s	remaining: 17.4s
325:	learn: 0.1357290	total: 8.41s	remaining: 17.4s
326:	learn: 0.1356458	total: 8.44s	remaining: 17.4s
327:	learn: 0.1355781	total: 8.47s	remaining: 17.4s
328:	learn: 0.1355102	total: 8.51s	remaining: 17.4s
329:	learn: 0.1354783	total: 8.54s	remaining: 17.3s
330:	learn: 0.1353860	total: 8.58s	remaining: 17.3s
331:	learn: 0.1352393	total: 8.62s	remaining: 17.3s
332:	learn: 0.1351982	total: 8.66s	remaining: 17.4s
333:	learn: 0.1351441	total: 8.7s	remaining: 17.3s
334:	learn: 0.1350582	total: 8.73s	remaining: 17.3s
335:	learn: 0.1349387	total: 8.77s	remaining: 17.3s
336:	learn: 0.1348860	total: 8.8s	remaining: 17.3s
337:	learn: 0.1348105	total: 8.85s	remaining: 17.3s
338:	learn: 0.1347430	total: 8.88s	remaining: 17.3s
339:	learn: 0.1347081	total: 8.92s	remaining: 17.3s
340:	learn: 0.1346628	total: 8.95s	remaining: 17.3s
341:	learn: 0.1345883	total: 8.98s	remaining: 17.3s
342:	learn: 0.1345516	total: 9.02s	remaining: 17.3s
343:	learn: 0.

484:	learn: 0.1270288	total: 12.7s	remaining: 13.5s
485:	learn: 0.1269210	total: 12.7s	remaining: 13.5s
486:	learn: 0.1268071	total: 12.8s	remaining: 13.4s
487:	learn: 0.1267686	total: 12.8s	remaining: 13.4s
488:	learn: 0.1267175	total: 12.8s	remaining: 13.4s
489:	learn: 0.1266741	total: 12.8s	remaining: 13.3s
490:	learn: 0.1266327	total: 12.8s	remaining: 13.3s
491:	learn: 0.1265797	total: 12.9s	remaining: 13.3s
492:	learn: 0.1265251	total: 12.9s	remaining: 13.2s
493:	learn: 0.1264682	total: 12.9s	remaining: 13.2s
494:	learn: 0.1264202	total: 12.9s	remaining: 13.2s
495:	learn: 0.1263876	total: 12.9s	remaining: 13.1s
496:	learn: 0.1263333	total: 12.9s	remaining: 13.1s
497:	learn: 0.1262918	total: 13s	remaining: 13.1s
498:	learn: 0.1262473	total: 13s	remaining: 13s
499:	learn: 0.1261980	total: 13s	remaining: 13s
500:	learn: 0.1261552	total: 13s	remaining: 13s
501:	learn: 0.1261054	total: 13.1s	remaining: 12.9s
502:	learn: 0.1260611	total: 13.1s	remaining: 12.9s
503:	learn: 0.1260099	tota

648:	learn: 0.1198764	total: 17.3s	remaining: 9.36s
649:	learn: 0.1198291	total: 17.3s	remaining: 9.33s
650:	learn: 0.1197845	total: 17.3s	remaining: 9.29s
651:	learn: 0.1197591	total: 17.4s	remaining: 9.26s
652:	learn: 0.1196839	total: 17.4s	remaining: 9.23s
653:	learn: 0.1196611	total: 17.4s	remaining: 9.2s
654:	learn: 0.1196218	total: 17.4s	remaining: 9.17s
655:	learn: 0.1196027	total: 17.4s	remaining: 9.14s
656:	learn: 0.1195641	total: 17.4s	remaining: 9.11s
657:	learn: 0.1195201	total: 17.5s	remaining: 9.07s
658:	learn: 0.1194884	total: 17.5s	remaining: 9.04s
659:	learn: 0.1194477	total: 17.5s	remaining: 9.01s
660:	learn: 0.1194104	total: 17.5s	remaining: 8.98s
661:	learn: 0.1193482	total: 17.5s	remaining: 8.95s
662:	learn: 0.1193149	total: 17.6s	remaining: 8.92s
663:	learn: 0.1192276	total: 17.6s	remaining: 8.89s
664:	learn: 0.1191930	total: 17.6s	remaining: 8.87s
665:	learn: 0.1191580	total: 17.6s	remaining: 8.84s
666:	learn: 0.1191154	total: 17.6s	remaining: 8.81s
667:	learn: 0

816:	learn: 0.1135857	total: 20.4s	remaining: 4.57s
817:	learn: 0.1135457	total: 20.4s	remaining: 4.54s
818:	learn: 0.1135139	total: 20.4s	remaining: 4.52s
819:	learn: 0.1134541	total: 20.5s	remaining: 4.49s
820:	learn: 0.1133928	total: 20.5s	remaining: 4.46s
821:	learn: 0.1133670	total: 20.5s	remaining: 4.44s
822:	learn: 0.1133315	total: 20.5s	remaining: 4.41s
823:	learn: 0.1132986	total: 20.5s	remaining: 4.38s
824:	learn: 0.1132610	total: 20.5s	remaining: 4.36s
825:	learn: 0.1132424	total: 20.6s	remaining: 4.33s
826:	learn: 0.1132177	total: 20.6s	remaining: 4.3s
827:	learn: 0.1131903	total: 20.6s	remaining: 4.28s
828:	learn: 0.1131370	total: 20.6s	remaining: 4.25s
829:	learn: 0.1130968	total: 20.6s	remaining: 4.23s
830:	learn: 0.1130735	total: 20.7s	remaining: 4.2s
831:	learn: 0.1130408	total: 20.7s	remaining: 4.17s
832:	learn: 0.1130203	total: 20.7s	remaining: 4.15s
833:	learn: 0.1129832	total: 20.7s	remaining: 4.12s
834:	learn: 0.1129446	total: 20.7s	remaining: 4.09s
835:	learn: 0.

985:	learn: 0.1080206	total: 23.5s	remaining: 334ms
986:	learn: 0.1079976	total: 23.5s	remaining: 310ms
987:	learn: 0.1079773	total: 23.5s	remaining: 286ms
988:	learn: 0.1079388	total: 23.6s	remaining: 262ms
989:	learn: 0.1079132	total: 23.6s	remaining: 238ms
990:	learn: 0.1078960	total: 23.6s	remaining: 214ms
991:	learn: 0.1078532	total: 23.6s	remaining: 190ms
992:	learn: 0.1078302	total: 23.6s	remaining: 167ms
993:	learn: 0.1078174	total: 23.7s	remaining: 143ms
994:	learn: 0.1077866	total: 23.7s	remaining: 119ms
995:	learn: 0.1077502	total: 23.7s	remaining: 95.1ms
996:	learn: 0.1077066	total: 23.7s	remaining: 71.3ms
997:	learn: 0.1076669	total: 23.7s	remaining: 47.5ms
998:	learn: 0.1076515	total: 23.7s	remaining: 23.8ms
999:	learn: 0.1076271	total: 23.8s	remaining: 0us
              precision    recall  f1-score   support

           0       0.91      0.98      0.95      8517
           1       0.48      0.13      0.21       912

    accuracy                           0.90      9429


In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_adasyn= GridSearchCV(estimator= cat_adasyn, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat_adasyn.fit(X_adasyn_train, y_adasyn_train)

print("Best Parameters:", gs_cat_adasyn.best_params_)
print("Best Precision Score:", gs_cat_adasyn.best_score_)

tuned_cat_adasyn= gs_cat_adasyn.best_estimator_
y_pred_tuned_cat_adasyn= tuned_cat_adasyn.predict(X_test)
y_pred_prob_tuned_cat_adasyn= tuned_cat_adasyn.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_adasyn, labels= tuned_cat_adasyn.classes_)
print(classification_report(y_test, y_pred_tuned_cat_adasyn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_adasyn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_adasyn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_adasyn))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.4883722	total: 18.7ms	remaining: 915ms
1:	learn: 0.3911062	total: 35.4ms	remaining: 849ms
2:	learn: 0.3344177	total: 49.9ms	remaining: 782ms
3:	learn: 0.2948157	total: 62.5ms	remaining: 719ms
4:	learn: 0.2632264	total: 75.7ms	remaining: 682ms
5:	learn: 0.2518986	total: 85.5ms	remaining: 627ms
6:	learn: 0.2257941	total: 98.2ms	remaining: 603ms
7:	learn: 0.2192017	total: 111ms	remaining: 580ms
8:	learn: 0.2146089	total: 122ms	remaining: 555ms
9:	learn: 0.2050464	total: 133ms	remaining: 532ms
10:	learn: 0.1955163	total: 145ms	remaining: 515ms
11:	learn: 0.1853910	total: 157ms	remaining: 498ms
12:	learn: 0.1774129	total: 169ms	remaining: 481ms
13:	learn: 0.1716878	total: 179ms	remaining: 462ms
14:	learn: 0.1695392	total: 190ms	remaining: 444ms
15:	learn: 0.1639607	total: 202ms	remaining: 429ms
16:	learn: 0.1628413	total: 213ms	remaining: 414ms
17:	learn: 0.1617274	total: 224ms	remaining: 398ms
18:	learn: 0.1612619	to

--- Visualisation ---

In [35]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_adasyn, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_adasyn, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_adasyn, zero_division= 0))
print('KNeighborsClassifier Precision:', precision_score(y_test, y_pred_knn_adasyn, zero_division= 0))
print('Tuned KNeighborsClassifier Precision:', precision_score(y_test, y_pred_tuned_knn_adasyn, zero_division= 0))
print('GaussianNB Precision:', precision_score(y_test, y_pred_nb_adasyn, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_adasyn, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_adasyn, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_adasyn, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_adasyn, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_adasyn, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_adasyn, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_adasyn, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_adasyn, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_adasyn, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_adasyn, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_adasyn, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_adasyn, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_adasyn, zero_division= 0))

Logistic Regression Precision: 0.2426208238728511
Decision Tree Precision: 0.23221343873517786
Tuned Decision Tree Precision: 0.30597014925373134
KNeighborsClassifier Precision: 0.21672555948174324
Tuned KNeighborsClassifier Precision: 0.24684431977559607
GaussianNB Precision: 0.16550594018844736
SVM Precision: 0.15486725663716813
Random Forest Precision: 0.36619718309859156
Tuned Random Forest Precision: 0.42696629213483145
AdaBoost Precision: 0.3776744186046512
Tuned AdaBoost Precision: 0.46862745098039216
GradientBoosting Precision: 0.42219020172910665
Tuned GradientBoosting Precision: 0.48214285714285715
XGB Precision: 0.45018450184501846
Tuned XGB Precision: 0.4619289340101523
LGBM Precision: 0.5340909090909091
Tuned LGBM Precision: 0.5361216730038023
CatBoost Precision: 0.4817813765182186
Tuned CatBoost Precision: 0.559322033898305


In [36]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_adasyn, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_adasyn, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_adasyn, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn_adasyn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn_adasyn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb_adasyn, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_adasyn, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_adasyn, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_adasyn, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_adasyn, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_adasyn, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_adasyn, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_adasyn, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_adasyn, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_adasyn, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_adasyn, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_adasyn, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_adasyn, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_adasyn, zero_division= 0)
                  }

adasyn_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
adasyn_precision= adasyn_precision.sort_values(by= 'Precision Score', ascending=False)
print(adasyn_precision)

                                    Model  Precision Score
18              Tuned CatBoost Precision:         0.559322
16                  Tuned LGBM Precision:         0.536122
15                        LGBM Precision:         0.534091
12      Tuned GradientBoosting Precision:         0.482143
17                    CatBoost Precision:         0.481781
10              Tuned AdaBoost Precision:         0.468627
14                   Tuned XGB Precision:         0.461929
13                         XGB Precision:         0.450185
8          Tuned Random Forest Precision:         0.426966
11            GradientBoosting Precision:         0.422190
9                     AdaBoost Precision:         0.377674
7                Random Forest Precision:         0.366197
2          Tuned Decision Tree Precision:         0.305970
4   Tuned KNeighborsClassifier Precision:         0.246844
0          Logistic Regression Precision:         0.242621
1                Decision Tree Precision:         0.2322

In [37]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_adasyn),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_adasyn),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_adasyn),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn_adasyn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn_adasyn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb_adasyn),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_adasyn),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_adasyn),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_adasyn),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_adasyn),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_adasyn),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_adasyn),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_adasyn),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_adasyn),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_adasyn),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_adasyn),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_adasyn),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_adasyn),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_adasyn)
                  }

adasyn_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
adasyn_accuracy= adasyn_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(adasyn_accuracy)

                                   Model  Accuracy Score
18              Tuned CatBoost Accuracy:        0.906247
16                  Tuned LGBM Accuracy:        0.905292
15                        LGBM Accuracy:        0.905186
17                    CatBoost Accuracy:        0.902323
12      Tuned GradientBoosting Accuracy:        0.902004
13                         XGB Accuracy:        0.900414
14                   Tuned XGB Accuracy:        0.900095
10              Tuned AdaBoost Accuracy:        0.899883
8          Tuned Random Forest Accuracy:        0.899141
7                Random Forest Accuracy:        0.895217
11            GradientBoosting Accuracy:        0.891823
6                          SVM Accuracy:        0.886732
2          Tuned Decision Tree Accuracy:        0.875703
9                     AdaBoost Accuracy:        0.875384
4   Tuned KNeighborsClassifier Accuracy:        0.864991
1                Decision Tree Accuracy:        0.845795
3         KNeighborsClassifier 

In [38]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_adasyn),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_adasyn),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_adasyn),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn_adasyn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn_adasyn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb_adasyn),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_adasyn),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_adasyn),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_adasyn),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_adasyn),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_adasyn),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_adasyn),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_adasyn),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_adasyn),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_adasyn),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_adasyn),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_adasyn),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_adasyn),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_adasyn)
                  }

adasyn_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
adasyn_auc= adasyn_auc.sort_values(by= 'AUC Score', ascending=False)
print(adasyn_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.851194
10              Tuned AdaBoost AUC:   0.848145
16                  Tuned LGBM AUC:   0.846907
15                        LGBM AUC:   0.846453
12      Tuned GradientBoosting AUC:   0.846410
14                   Tuned XGB AUC:   0.844625
11            GradientBoosting AUC:   0.844401
18              Tuned CatBoost AUC:   0.844038
17                    CatBoost AUC:   0.843178
9                     AdaBoost AUC:   0.837987
13                         XGB AUC:   0.834493
8          Tuned Random Forest AUC:   0.832723
7                Random Forest AUC:   0.820868
5                   GaussianNB AUC:   0.801071
3         KNeighborsClassifier AUC:   0.739399
2          Tuned Decision Tree AUC:   0.718136
6                          SVM AUC:   0.698870
4   Tuned KNeighborsClassifier AUC:   0.620761
1                Decision Tree AUC:   0.582264
